In [ ]:
from itertools import product
import pprint

#%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy import interpolate

# Constants
df1 = pd.read_csv('aff_elementonly.txt', header = None, delimiter = ',')
df2 = pd.read_csv('aff_parmonly.txt', header = None, delim_whitespace=True)
Scattering_Factors = df2.T
Scattering_Factors.columns = df1[0]
Scattering_Factors

In [ ]:
def load_atom_positions(filename):
    """
    Load data from .xyz file
    no header
    """
    atom_positions_df = pd.read_table(
        filepath_or_buffer=filename,
        header=None,
        index_col=0,
        delim_whitespace=True
    ).transpose()
    # maybe atoms do not need to be numbered
    #atom_positions_df.columns = [f"{atom}_{n}" for n, atom in enumerate(atom_positions_df.columns)]
    atom_positions_df.index = ["x", "y", "z"]
    return atom_positions_df

In [ ]:
# just a test
from io import StringIO
salt_xyz = StringIO(
"""\
Na 0 0 1
Cl 0 1 0
Na 1 0 0
""")
salt_atom_positions_df = load_atom_positions(filename=salt_xyz)
salt_atom_positions_df

In [ ]:
def create_atom_distance_matrix(atom_positions_df):
    #print("atom_positions_df.T.to_numpy()")
    #print(atom_positions_df.T.to_numpy())
    atom_distance_matrix = cdist(
        atom_positions_df.T.to_numpy(),
        atom_positions_df.T.to_numpy()
    )
    #print("atom_distance_matrix")
    #print(atom_distance_matrix)
    atom_distance_matrix_df = pd.DataFrame(
        data=atom_distance_matrix,
        columns=atom_positions_df.columns,
        index=atom_positions_df.columns
    )
    #print(f'Number of Atom Pairs: {atom_distance_matrix_df.size}')
    # set index name to None, otherwise it is "0" and that looks
    #   odd when the dataframe is printed
    atom_distance_matrix_df.index.name = None
    return atom_distance_matrix_df

In [ ]:
# just a test
salt_atom_distance_matrix_df = create_atom_distance_matrix(salt_atom_positions_df)
salt_atom_distance_matrix_df

In [ ]:
# a little test
a1 = 10.0
b1 = 2.0
q1 = 0.1
f1 = a1 * np.exp( -b1 * ((q1/(4*np.pi)))**2)
f1

In [ ]:
g1 = np.exp(-b1)
f1 = a1 * (g1 ** (((q1/(4*np.pi)))**2))
f1

In [ ]:
def calculateIqData(atom_distance_matrix_df, qmin=0.6, qmax=20, qstep=0.05):
    # atom_element looks like
    #   ['O', 'Co', 'O', 'O', 'O', 'O', 'O', 'Co', 'Co',...]
    #atom_element = np.array([
    #    element_number.split('_')[0]
    #    for element_number
    #    in atom_distance_matrix_df.columns
    #])
    atom_element = atom_distance_matrix_df.columns
    print("atom_element")
    print(atom_element)

    # set(atom_element) looks like {'O', 'Co'}
    unique_elements = set(atom_element)
    print(f"unique elements: {unique_elements}")

    atom_distance_matrix = atom_distance_matrix_df.to_numpy()
    Iq_sum_list = []

    # we can allocate this vector once and reuse it
    # Fi needs shape (1, atoms count) so we can calculate an outer product
    #   Fi.T * Fi
    Fi = np.zeros((1, len(atom_element)), dtype=np.float64)
    print("initial Fi")
    print(Fi)

    # loop on q
    q_range = np.arange(qmin, qmax, qstep)
    print(f"q_range: {q_range}")
    for q in q_range:
        # can we calculate some of this ahead?
        for element in unique_elements:
            scattering_values = Scattering_Factors[element]
            fi1 = scattering_values[0]*np.exp(-scattering_values[1]*((q/(4*np.pi))**2))
            fi2 = scattering_values[2]*np.exp(-scattering_values[3]*((q/(4*np.pi))**2))
            fi3 = scattering_values[4]*np.exp(-scattering_values[5]*((q/(4*np.pi))**2)) 
            fi4 = scattering_values[6]*np.exp(-scattering_values[7]*((q/(4*np.pi))**2))
            fic = scattering_values[8]

            #print("atom_element == ", element)
            #print(atom_element == element)
            Fi[0, atom_element == element] = fi1 + fi2 + fi3 + fi4 + fic
        
        #print(f"Fi:")
        #print(Fi)
        #print(np.shape(Fi))
        FiFj = Fi.T * Fi
        #print("FiFj")
        #print(FiFj)

        if q > 0.0:
            # the next line will cause a warning like this:
            #   ../site-packages/ipykernel_launcher.py:???: 
            #   RuntimeWarning: invalid value encountered in true_divide
            # but this is not an error, it tells us the sin_term_matrix has
            # NaN on the diagonal which will be corrected on the following line
            # this line is the bottleneck when the number of atoms gets large
            sin_term_matrix = np.sin(q*atom_distance_matrix) / (q*atom_distance_matrix)
            # set the diagonal elements to 1.0
            sin_term_matrix[np.diag_indices(sin_term_matrix.shape[0])] = 1.0
            #print("sin_term_matrix")
            #print(sin_term_matrix)
        elif q == 0.0:
            sin_term_matrix = np.eye_like(atom_distance_matrix)
        else:
            # q is less than 0.0
            raise ValueError(f"q is less than zero: {q}")

        Iq = FiFj * sin_term_matrix

        # sum Iq for each pair only once
        #Iq_sum_list.append(np.sum(Iq[np.triu_indices(Iq.shape[0])]))

        # sum Iq for each pair twice, except for "self" pairs such as (O_0, O_0)
        # (pairs from the diagonal of the distance matrix)
        Iq_sum_list.append(np.sum(Iq))

    #print("Iq.shape")
    #print(Iq.shape)
    qIq = np.column_stack((
        q_range,
        Iq_sum_list
    ))
    #print("qIq")
    #print(qIq)
    
    return qIq, Fi

In [ ]:
def g_calculateIqData(atom_distance_matrix_df, qmin=0.6, qmax=20, qstep=0.05):
    import cupy as cp
    # atom_element looks like
    #   ['O', 'Co', 'O', 'O', 'O', 'O', 'O', 'Co', 'Co',...]
    #atom_element = np.array([
    #    element_number.split('_')[0]
    #    for element_number
    #    in atom_distance_matrix_df.columns
    #])
    atom_element = atom_distance_matrix_df.columns
    print("atom_element")
    print(atom_element)

    # set(atom_element) looks like {'O', 'Co'}
    unique_elements = set(atom_element)
    print(f"unique elements: {unique_elements}")

    atom_distance_matrix = cp.asarray(atom_distance_matrix_df.to_numpy())
    Iq_sum_list = []
    
    # we can allocate this vector once and reuse it
    # Fi needs shape (1, atoms count) so we can calculate an outer product
    #   Fi.T * Fi
    Fi = cp.zeros((1, len(atom_element)), dtype=np.float64)
    print("initial Fi")
    print(Fi)

    # calculate some vectors that never change
    Si1 = cp.zeros((1, len(atom_element)), dtype=np.float64)
    Gi1 = cp.zeros((1, len(atom_element)), dtype=np.float64)
    Si2 = cp.zeros((1, len(atom_element)), dtype=np.float64)
    Gi2 = cp.zeros((1, len(atom_element)), dtype=np.float64)
    Si3 = cp.zeros((1, len(atom_element)), dtype=np.float64)
    Gi3 = cp.zeros((1, len(atom_element)), dtype=np.float64)
    Si4 = cp.zeros((1, len(atom_element)), dtype=np.float64)
    Gi4 = cp.zeros((1, len(atom_element)), dtype=np.float64)
    Sic = cp.zeros((1, len(atom_element)), dtype=np.float64)
    for element in unique_elements:
        scattering_values = cp.asarray(Scattering_Factors[element])
        Si1[0, atom_element == element] = scattering_values[0]     
        Gi1[0, atom_element == element] = cp.exp( -scattering_values[1] * (1.0/(4.0 * cp.pi))**2 )
        Si2[0, atom_element == element] = scattering_values[2]     
        Gi2[0, atom_element == element] = cp.exp( -scattering_values[3] * (1.0/(4.0 * cp.pi))**2 )
        Si3[0, atom_element == element] = scattering_values[4]     
        Gi3[0, atom_element == element] = cp.exp( -scattering_values[5] * (1.0/(4.0 * cp.pi))**2 )
        Si4[0, atom_element == element] = scattering_values[6]     
        Gi4[0, atom_element == element] = cp.exp( -scattering_values[7] * (1.0/(4.0 * cp.pi))**2 )
        Sic[0, atom_element == element] = scattering_values[8]     

    sin_matrix_diagonal_index = cp.diag_indices_from(atom_distance_matrix)

    # loop on q
    q_range = np.arange(qmin, qmax, qstep)
    print(f"q_range: {q_range}")
    for q in q_range:        
        Fi = Si1 * cp.power(Gi1, q**2) + Si2 * cp.power(Gi2, q**2) + Si3 * cp.power(Gi3, q**2) + Si4 * cp.power(Gi4, q**2) +  Sic
        #print(f"Fi:")
        #print(Fi)
        #print(np.shape(Fi))
        FiFj = Fi.T * Fi
        #print("FiFj")
        #print(FiFj)

        if q > 0.0:
            # the next line will cause a warning like this:
            #   ../site-packages/ipykernel_launcher.py:???: 
            #   RuntimeWarning: invalid value encountered in true_divide
            # but this is not an error, it tells us the sin_term_matrix has
            # NaN on the diagonal which will be corrected on the following line

            #sin_term_matrix = np.sin(q*atom_distance_matrix) / (q*atom_distance_matrix)
            q_atom_distance_matrix = q*atom_distance_matrix
            sin_term_matrix = cp.sin(q_atom_distance_matrix) / q_atom_distance_matrix

            # set the diagonal elements to 1.0
            sin_term_matrix[sin_matrix_diagonal_index] = 1.0
            #print("sin_term_matrix")
            #print(sin_term_matrix)
        elif q == 0.0:
            sin_term_matrix = cp.eye_like(atom_distance_matrix)
        else:
            raise ValueError(f"q is less than zero: {q}")

        Iq = FiFj * sin_term_matrix

        # sum Iq for each pair only once
        #Iq_sum_list.append(np.sum(Iq[np.triu_indices(Iq.shape[0])]))

        # sum Iq for each pair twice, except for "self" pairs such as (O_0, O_0)
        # (pairs from the diagonal of the distance matrix)
        Iq_sum_list.append(cp.sum(Iq))

    #print("Iq.shape")
    #print(Iq.shape)
    qIq = np.column_stack((
        q_range,
        [s.get() for s in Iq_sum_list]
    ))
    #print("qIq")
    #print(qIq)
    
    return qIq, Fi.get()

In [ ]:
# just a test
calculateIqData(atom_distance_matrix_df=salt_atom_distance_matrix_df, qmin=0.5, qmax=0.6, qstep=0.05)

In [ ]:
# just a test
g_calculateIqData(atom_distance_matrix_df=salt_atom_distance_matrix_df, qmin=0.5, qmax=0.6, qstep=0.05)

In [ ]:
def getQIq(file, qmin=0.6, qmax=30, qstep=0.02, auto_save=False):
    atom_positions_df = load_atom_positions(filename=file)
    
    atom_distance_matrix_df = create_atom_distance_matrix(atom_positions_df)
    #print("atom_distance_matrix_df", atom_distance_matrix_df.shape)
    #print(atom_distance_matrix_df[:10])

    qIq,Fi = calculateIqData(
        atom_distance_matrix_df,
        qmin=qmin, qmax=qmax, qstep=qstep
    )
    if auto_save:
        np.savetxt(file.split('.')[0]+'_QIq_Restuls.txt',qIq)
    return qIq, Fi

In [ ]:
%load_ext line_profiler

In [ ]:
atom_positions_df = load_atom_positions(filename="5IrC_r5a-1Ir.xyz")
atom_distance_matrix_df = create_atom_distance_matrix(atom_positions_df)
%lprun -f calculateIqData calculateIqData(atom_distance_matrix_df=atom_distance_matrix_df, qmin=0.01, qmax=30, qstep=0.01)

In [ ]:
%lprun -f g_calculateIqData g_calculateIqData(atom_distance_matrix_df=atom_distance_matrix_df, qmin=0.01, qmax=30, qstep=0.01)

In [ ]:
atom_positions_df = load_atom_positions(filename="Ni(OH)2-109391-ICSD-10x10x1.xyz")
atom_distance_matrix_df = create_atom_distance_matrix(atom_positions_df)
#%lprun -f calculateIqData calculateIqData(atom_distance_matrix_df=atom_distance_matrix_df, qmin=0.01, qmax=30, qstep=0.01)

In [ ]:
atom_positions_df = load_atom_positions(filename="Ni(OH)2-109391-ICSD-10x10x10.xyz")
atom_distance_matrix_df = create_atom_distance_matrix(atom_positions_df)
%lprun -f g_calculateIqData g_calculateIqData(atom_distance_matrix_df=atom_distance_matrix_df, qmin=0.01, qmax=30, qstep=0.01)

In [ ]:
qIq, Fi = getQIq(file="5IrC_r5a-1Ir.xyz", qmin=0.01, qmax=30, qstep=0.01, auto_save=False)

In [ ]:
plt.plot(qIq[:,0],np.log10(qIq[:,1]))

# Sq Calculation below
### Modified the above code to have sq also calculated at the same time as Iq

In [ ]:
def calculateSqData(atom_distance_matrix_df, qmin = 0.6, qmax =20 ,qstep =0.05 ):
    # atom_element looks like
    #   ['O', 'Co', 'O', 'O', 'O', 'O', 'O', 'Co', 'Co',...]
    atom_element = np.array([
        element_number.split('_')[0]
        for element_number
        in atom_distance_matrix_df.columns
    ])
    
    N = len(atom_element)

    # set(atom_element) looks like {'O', 'Co'}
    unique_elements = np.array(list(set(atom_element)))
    
    unique_elements_dict = {unique_elem:np.count_nonzero(atom_element == unique_elem) for unique_elem in unique_elements} 


    atom_distance_matrix = atom_distance_matrix_df.to_numpy()
    Iq_sum_list = []
    Fs_list = []
    Fs_sqr_list = []
    
    # loop on q rather than atom pairs
    q_range = np.arange(qmin, qmax, qstep)
    
    for  q in q_range:
        Fi = np.full_like(atom_element, fill_value=np.nan, dtype=np.float64)
        Fi_sq = np.full_like(unique_elements, fill_value=np.nan, dtype=np.float64)
        #print(f"Fi ; {Fi}" )
        
        for element in atom_element:
            #print(f"element = {element}")
            scattering_values = Scattering_Factors[element]
            fi1 = scattering_values[0]*np.exp(-scattering_values[1]*((q/(4*np.pi))**2))
            fi2 = scattering_values[2]*np.exp(-scattering_values[3]*((q/(4*np.pi))**2))
            fi3 = scattering_values[4]*np.exp(-scattering_values[5]*((q/(4*np.pi))**2)) 
            fi4 = scattering_values[6]*np.exp(-scattering_values[7]*((q/(4*np.pi))**2))
            fic = scattering_values[8]

            #print(f"unique_element == {element}")
            Fi[atom_element == element] = fi1 + fi2 + fi3 + fi4 + fic
            Fi_sq[unique_elements == element] = fi1 + fi2 + fi3 + fi4 + fic
        
        Fs_sqr_list.append(np.dot(np.square(Fi_sq),list(unique_elements_dict.values()))*(1/N))
        Fs_list.append(np.dot(Fi_sq,list(unique_elements_dict.values()))*(1/N))
        
        #sq_df = pd.DataFrame(Fi_sq_list, columns=unique_elements_dict.keys())
        
        
        
        Fi = Fi[np.newaxis, :]
        #print(np.shape(Fi))
        FiFj = Fi.T * Fi
        #print("FiFj")
        #print(FiFj)
        # the next line will cause a warning like this:
        #   ../site-packages/ipykernel_launcher.py:???: 
        #   RuntimeWarning: invalid value encountered in true_divide
        # but this is not an error, it tells us the sin_term_matrix has
        # NaN on the diagonal which will be corrected on the following line
        sin_term_matrix = np.sin(q*atom_distance_matrix) / (q*atom_distance_matrix)
        # set the diagonal elements to 1.0
        sin_term_matrix[np.diag_indices(sin_term_matrix.shape[0])] = 1.0
        #print("sin_term_matrix")
        #print(sin_term_matrix)

        Iq = FiFj * sin_term_matrix

        # sum Iq for each pair only once
        #Iq_sum_list.append(np.sum(Iq[np.triu_indices(Iq.shape[0])]))

        # sum Iq for each pair twice, except for "self" pairs such as (O_0, O_0)
        # (pairs from the diagonal of the distance matrix)
        Iq_sum_list.append(np.sum(Iq))
        
    #print(sq_data.head())
    #print("Iq.shape")
    #print(Iq.shape)
    
    sq = np.ones(q_range.shape) + (Iq_sum_list/(N*np.square(Fs_list))) - (Fs_sqr_list/np.square(Fs_list))
    fq = (sq - np.ones(q_range.shape))*q_range
    
    sq_dataframe = pd.DataFrame(np.column_stack([q_range,Iq_sum_list]),
                           columns= ['q', 'Iq'])
    sq_dataframe['Sq'] = sq
    sq_dataframe['Fq'] = fq
    
    qIq = np.column_stack((
        q_range,
        Iq_sum_list
    ))
    #print("qIq")
    #print(qIq)
    return sq_dataframe

def getSQIq(file = '28108-ICSDNi23_SolX.xyz', qmin = 0.6, qmax =30 ,qstep =0.02, auto_save = False):
    atom_positions, atom_positions_df = load_data(filename = file)
    
    atom_distance_matrix_df = create_atom_distance_matrix(atom_positions_df)
    #print("atom_distance_matrix_df", atom_distance_matrix_df.shape)
    #print(atom_distance_matrix_df[:10])

    sq_dataframe = calculateSqData(
        atom_distance_matrix_df,
        qmin=qmin, qmax=qmax, qstep=qstep
    )
    
    plt.plot(sq_dataframe['q'],sq_dataframe['Sq'])
    plt.plot(sq_dataframe['q'],sq_dataframe['Fq'])
    plt.xlabel('q')
    plt.ylabel('Sq')
    plt.title('q vs Sq')
    
    if auto_save:
        sq_dataframe.to_csv(file.split('.')[0]+'_Sq_Restuls.csv', index = False)
    return sq_dataframe


def getGr(q,fq,rmin=0,rmax= 100,rstep = 0.01,extrapolate_method = 'scipy'):

    #rmin and rmax are not for saved data
    #they are for rstep (x-axis) after IFFT, please see the min_x = int(np.ceil((rmin) / rstep))   
    #max_x = int(np.floor((rmax) / rstep)) + 1  
   
    qstep = q[1] - q[0]
    #print("beforecalfq =", len(fq))
    #print("qstep = ", qstep)

    datapoint = int (q[1]/qstep)
    #print("datapoint = ", datapoint)
    before_q = np.zeros(datapoint)  #calqx = before_q
    before_fq = np.zeros(datapoint)   #calfqx  = before_fq

    #linear extrapolation between 0 to starting q[0] from the loaded fq data.
    #For inverse fft, fq data point must be started at q=0 
    # if q start from 0, adding points from 0 to q[0] is not necessary
    # option1: linear extrapolation from 0 to fq[0]
    if q[0] > 0:
        if extrapolate_method == 'linear': #linear interpolation
            for x in range (datapoint):
                qi = x*qstep
                before_q[x] = qi
                before_fq[x] = fq[1]/q[1]*qi
        if extrapolate_method == 'scipy':  #linear extrapolation
            #extrapolation function
            f = interpolate.interp1d(q, fq, fill_value = "extrapolate")
            for x in range(datapoint):
                before_q[x] = x*qstep
                before_fq[x] = f(before_q[x])

    #calq, calfq를 위에서 만들어진 calqx, calfqx와 데이타를 합친다.
    #append calqx and calq and then save calqx, calfqx together
    q_final = np.append(before_q, q)
    fq_final = np.append(before_fq, fq)

    #making x-axis data set after IFFT
    #np.ceil does round at first decimal point. 0.5 --> 1, 0.4 --> 0
    #np.floor cuts decimal point and does not round 0.5--> 0, 1.5-->1
    #adding np.ceil and np.floor is safe to get xout. 
    #without np.ceil and np.floor, when rmin = 0.01 and rstep = 0.19,
    #lostep_nonpceil 0.05263157894736842, histep_nonpfloor 527.3157894736842
    #flooting number may cause issue at xout by arange function.
    #"+ 1" is required at histep because the last datapoint must be matched with the last datapoint at histep
    #"np.arange(min_x, max_x) * rstep" is for matching "step of xout data" with "rstep".
    min_x = int(np.ceil((rmin) / rstep))   
    max_x = int(np.floor((rmax) / rstep)) + 1  

    #make x-axis (real space, r(Angstrom)) data set after IFFT, step of x_axis si equla to rstep. 
    x_axis = np.arange(min_x, max_x) * rstep
    #print('x_axis = ', x_axis)

    #realationship between qmax (=qmaxrstep) vs. rstep --> rstep = np.pi/qmax.
    #refer to page 101 in "Underneath the bragg Peaks 2nd edition"
    qmaxrstep = np.pi / rstep

    #print('qmaxrstep = ', qmaxrstep)
    nin = len(q_final)
    #print('nin = ', nin)
    nbase = max(nin, max_x, qmaxrstep / qstep)
    #print('max_x = ', max_x)
    #print('qmaxrstep / qstep = ', qmaxrstep / qstep)
    #print('nbase =', nbase)
    ####==========================below ?????????????
    #nbaselog2 = np.log2(nbase)
    #print('nbaselog2 = ', nbaselog2)
    # refer https://github.com/diffpy/libdiffpy/blob/1bfd8f8ae9ec17d2cc0fdf32d6af5e75d10c3b9d/src/diffpy/srreal/PDFUtils.cpp#L45-L84%200

    nlog2 = int(np.ceil(np.log2(nbase)))
    #print('nlog2 = ', nlog2)
    nout = 2 ** nlog2
    #print('nout = ', nout)
    qmaxdb = 2 * nout * qstep
    #print('qmaxdb = ', qmaxdb)

    #####==========================above ????????????????

    yindb = np.concatenate((fq_final, np.zeros(2 * nout - nin)))
    #np.savetxt('yindb.txt', yindb) #what's this?
    print('yindb = ', yindb)
    #Gr calculation by inverse fft (IFFT)
    cyoutdb = np.fft.ifft(yindb) * 2 / np.pi * qmaxdb
    youtdb = np.imag(cyoutdb)
    xstepfine = 2 * np.pi / qmaxdb
    xoutfine = np.arange(nout) * xstepfine
    youtfine = youtdb[:nout]
    
    return xoutfine, youtfine

In [ ]:
%time qIq = getSQIq(file='5IrC_r5a-1Ir.xyz', qmin=0.01, qmax=30, qstep=0.01, auto_save=True)

In [ ]:
def getSQIqGr(file = '28108-ICSDNi23_SolX.xyz', qmin = 0.6, qmax =30 ,qstep =0.02,
              rmin=0,rmax= 100,rstep = 0.01, auto_save = False):
    
    atom_positions, atom_positions_df = load_data(filename = file)
    
    atom_distance_matrix_df = create_atom_distance_matrix(atom_positions_df)
    #print("atom_distance_matrix_df", atom_distance_matrix_df.shape)
    #print(atom_distance_matrix_df[:10])

    SqIqGr_dataframe = calculateSqData(
        atom_distance_matrix_df,
        qmin=qmin, qmax=qmax, qstep=qstep
    )
    
    r, gr = getGr(SqIqGr_dataframe['q'],SqIqGr_dataframe['Fq'],rmin=rmin,rmax= rmax,rstep = rstep,
                  extrapolate_method = 'linear')  #'linear' or 'scipy'
    
    print(f" r length ={len(r)}, pd series ;  {len(pd.Series(r))}")
    print(f" r last ={r[-1]}")
    print(f"q length ={len(SqIqGr_dataframe['q'])}")
    
    Gr_dataframe = pd.DataFrame(columns=['r', 'Gr'])
    
    Gr_dataframe['r'] = pd.Series(r)
    Gr_dataframe['Gr'] = pd.Series(gr)
          
    
    SqIqGr_dataframe = pd.concat([Gr_dataframe, SqIqGr_dataframe], axis=1)
          
    fig, (ax1, ax2) = plt.subplots(2, 1)
    # make a little extra space between the subplots
    fig.subplots_adjust(hspace=0.5)
    

    ax1.plot(SqIqGr_dataframe['q'],SqIqGr_dataframe['Sq'], label = 'S(q)')
    ax1.plot(SqIqGr_dataframe['q'],SqIqGr_dataframe['Fq'], label = 'F(q)')
    ax1.set_xlabel('q')
    ax1.set_ylabel('Sq')
    ax1.set_title('q vs Sq vs Fq')
    ax1.legend()
    
    ax2.plot(SqIqGr_dataframe['r'],SqIqGr_dataframe['Gr'], 'r')
    ax2.set_xlabel('r')
    ax2.set_ylabel('Gr')
    ax2.set_title('r vs Gr')
    plt.xlim(rmin, rmax)
    
    if auto_save:
        SqIqGr_dataframe.to_csv(file.split('.')[0]+'_IqSqGr_Restuls.csv', index = False)
    return SqIqGr_dataframe

In [ ]:
df = getSQIqGr(file = '5IrC_r5a-1Ir.xyz', qmin = 0.01, qmax =30, qstep =0.01, 
               rmin=0, rmax= 15, rstep = 0.01, auto_save = True)